<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/dune/Cypher_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 KB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/01/b1/f0fe9bc5f59d54bd0ff74afdfdc58911aaab53315f2ed40087b018eb54f8/neo4j-5.4.0.tar.gz#sha256=50293f716412cc8a0fc87c364b0da105aa439859d28ffa5c89f5ca0d44514049 (from https://pypi.org/simple/neo4j/) (requires-python:>=3.7): Requested unknown from https://files.pythonhosted.org/packages/01/b1/f0fe9bc5f59d54bd0ff74afdfdc58911aaab53315f2ed40087b018eb54f8/neo4j-5.4.0.tar.gz#sha256=50293f716412cc8a0fc87c364b0da105aa439859d28ffa5c89f5ca0d44514049 has inconsistent name: filename has 'neo4j', but metadata has 'unknown'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.8/157.8 KB 16.5 MB/s eta 0:00

In [3]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase

host = 'neo4j+s://f4461036.databases.neo4j.io'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [4]:
run_query("""
CREATE CONSTRAINT IF NOT EXISTS FOR (c:Character) REQUIRE c.name IS UNIQUE;
""")

""


In [5]:
run_query("""
LOAD CSV WITH HEADERS FROM "https://bit.ly/3kbU21V" AS row
CALL {
  WITH row
  MERGE (c:Character {name:row.Character})
  SET c += apoc.map.clean(row, ["name", "Detail", "to", "relationship_type", "House_Allegiance"], [])
  WITH c, row
  CALL apoc.create.addLabels(c, [row.House_Allegiance]) YIELD node
  CALL {
    WITH row, c
    WITH row, c
    WHERE row.to IS NOT NULL
    MERGE (c1:Character {name: row.to})
    WITH row, c, c1
    CALL apoc.merge.relationship(c, toUpper(row.relationship_type), {}, {}, c1)
    YIELD rel
    RETURN count(*) AS count
  }
  RETURN count(*) AS finalCount
} IN TRANSACTIONS OF 10 ROWS
RETURN count(*)
""")

,count(*)
0,107


In [5]:
run_query("""
MATCH (c:Fremen|Harkonnen)
RETURN count(*)
""")

,count(*)
0,16


In [6]:
run_query("""
MATCH (c:Character&Harkonnen)
RETURN count(*)
""")

,count(*)
0,15


In [7]:
run_query("""
MATCH (c:Character&!Harkonnen)
RETURN count(*)
""")

,count(*)
0,64


In [8]:
run_query("""
MATCH (c:!(Fremen|Harkonnen))
RETURN count(*)
""")

,count(*)
0,63


In [10]:
run_query("""
MATCH (c:Character WHERE c.Culture STARTS WITH "Z" AND c.Died IS NOT NULL)
RETURN c.name AS character
""")

,character
0,Ali Ben Ohashi
1,Maometh


In [11]:
run_query("""
MATCH p=(c:Character WHERE c.Culture = "Bene Gesserit")-[r:ALLIES|FAMILY]-()
RETURN [n in nodes(p) | n.name] AS pairs, type(r) AS results
""")

,pairs,results
0,"[Anirul Corrino, Shaddam Corrino IV]",FAMILY
1,"[Jessica Atreides, Paul Atreides]",FAMILY
2,"[Jessica Atreides, Paul Atreides]",FAMILY


In [12]:
run_query("""
MATCH p=(c:Character WHERE c.Culture = "Fremen")-[r:!(ALLIES|FAMILY)]-()
RETURN [n in nodes(p) | n.name] AS pairs, type(r) AS results
""")

,pairs,results
0,"[Chani Kynes, Paul Atreides]",MARRIAGE
1,"[Geoff, Jamis]",ENEMIES
2,"[Jamis, Geoff]",ENEMIES
3,"[Stilgar, Tharthar]",MARRIAGE
4,"[Liet Kynes, Uliet]",ENEMIES
5,"[Mitha, Pardot Kynes]",MARRIAGE


In [12]:
run_query("""
MATCH p=(c:Character WHERE c.Culture = "Fremen" AND c.Died IS NOT NULL)-[r:!(ALLIES|FAMILY)]-()
RETURN [n in nodes(p) | n.name] AS pairs, type(r) AS results
""")

,pairs,results
0,"[Jamis, Geoff]",ENEMIES
1,"[Liet Kynes, Uliet]",ENEMIES


In [13]:
run_query("""
MATCH (c:Character)
WHERE exists { (c)-[:FAMILY]-() }
RETURN count(*) AS count
""")

,count
0,27


In [14]:
run_query("""
MATCH (c:Character)
WHERE exists { MATCH (c)-[:FAMILY]-(t)
               WHERE (t)-[:ALLIES]-() AND (t)-[:MARRIAGE]-() }
RETURN count(*) AS count
""")

,count
0,5


In [18]:
run_query("""
MATCH (c:Character)
RETURN c.name AS character,
       count{ (c)--() } AS degree
ORDER BY degree DESC
LIMIT 5
""")

,character,degree
0,Paul Atreides,16
1,Leto Atreides I,15
2,Stilgar,7
3,Vladimir Harkonnen,6
4,Shaddam Corrino IV,6


In [15]:
run_query("""
MATCH (c:Character)
WHERE count{ (c)--() } > 2
RETURN count(*) AS count
""")

,count
0,11
